为防止在服务器断电时，不至于损失数天的计算结果，可以定期保存中间结果

加载和保存张量

In [1]:
import torch
from torch import nn 
from torch.nn import functional as F 

x = torch.arange(4)
torch.save(x, 'x-file')

In [2]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [4]:
# 存储一个张量列表，再读回内存
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [5]:
# 读取或写入从字符串映射到张量的字典
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [7]:
torch.save(net.state_dict(), 'mlp.params')

In [ ]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()  # 设置为评估模式，保持参数不变，不会更新参数 ———— 便于下面测试

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [10]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

练习

In [11]:
# 如何保存和加载模型的某层参数
torch.save(net.hidden.state_dict(), 'mlp.hidden.params')
clone = MLP()
clone.hidden.load_state_dict(torch.load('mlp.hidden.params'))
print(clone.hidden.weight == net.hidden.weight)

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])


In [12]:
# 如何同时保存网络架构和参数
# pt 即pytorch
model = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
torch.save(model, 'model.pt')
m = torch.load('model.pt')
print(m)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)
